# 文件读写

写入测试文件：

In [1]:
%%writefile test.txt
this is a test file.
hello world!
python is good!
today is a good day.

Writing test.txt


## 读文件

使用 `open` 函数或者 `file` 函数来读文件，使用文件名的字符串作为输入参数：

In [2]:
f = open('test.txt')

默认以读的方式打开文件，如果文件不存在会报错。

可以使用 `read` 方法来读入文件中的所有内容：

In [3]:
text = f.read()
print (text)

this is a test file.
hello world!
python is good!
today is a good day.



也可以按照行读入内容，`readlines` 方法返回一个列表，每个元素代表文件中每一行的内容：

In [4]:
f = open('test.txt')
lines = f.readlines()
print (lines)

['this is a test file.\n', 'hello world!\n', 'python is good!\n', 'today is a good day.\n']


使用完文件之后，需要将文件关闭。

In [6]:
f.close()

事实上，我们可以将 `f` 放在一个循环中，得到它每一行的内容：

In [5]:
f = open('test.txt')
for line in f:
    print (line)
f.close()

this is a test file.

hello world!

python is good!

today is a good day.



删除刚才创建的文件：

In [6]:
import os
os.remove('test.txt')

## 写文件

我们使用 `open` 函数的写入模式来写文件：

In [7]:
f = open('myfile.txt', 'w')
f.write('hello world!')
f.close()

使用 `w` 模式时，如果文件不存在会被创建，我们可以查看是否真的写入成功：

In [8]:
print (open('myfile.txt').read())

hello world!


如果文件已经存在， `w` 模式会覆盖之前写的所有内容：

In [9]:
f = open('myfile.txt', 'w')
f.write('another hello world!')
f.close()
print (open('myfile.txt').read())

another hello world!


除了写入模式，还有追加模式 `a` ，追加模式不会覆盖之前已经写入的内容，而是在之后继续写入：

In [10]:
f = open('myfile.txt', 'a')
f.write('... and more')
f.close()
print (open('myfile.txt').read())

another hello world!... and more


写入结束之后一定要将文件关闭，否则可能出现内容没有完全写入文件中的情况。

还可以使用读写模式 `w+`：

In [11]:
f = open('myfile.txt', 'w+')
f.write('hello world!')
f.seek(6)
print (f.read())
f.close()

world!


这里 `f.seek(6)` 移动到文件的第6个字符处，然后 `f.read()` 读出剩下的内容。

In [12]:
import os
os.remove('myfile.txt')

## 二进制文件

二进制读写模式 b：

In [23]:
import os
f = open('binary.bin', 'wb')
f.write(os.urandom(16))  # 生成指定长度的随机字节串。这个方法是用于加密学的。
f.close()

f = open('binary.bin', 'rb')
print (repr(f.read()))
f.close()

b'\x9e\xd9\xbf\xf0\x14\xbb\xc9\xbc\xa7\xd4\x02\xd2\x1e\xc7\xe5\x1b'


In [24]:
import os
os.remove('binary.bin')

## 换行符

不同操作系统的换行符可能不同：

- `\r`
- `\n`
- `\r\n`

使用 `U` 选项，可以将这三个统一看成 `\n` 换行符。

## 关闭文件

在**Python**中，如果一个打开的文件不再被其他变量引用时，它会自动关闭这个文件。

所以正常情况下，如果一个文件正常被关闭了，忘记调用文件的 `close` 方法不会有什么问题。

关闭文件可以保证内容已经被写入文件，而不关闭可能会出现意想不到的结果：

In [25]:
f = open('newfile.txt','w')
f.write('hello world')
g = open('newfile.txt', 'r')
print (repr(g.read()))

''


虽然这里写了内容，但是在关闭之前，这个内容并没有被写入磁盘。

使用循环写入的内容也并不完整：

In [47]:
f = open('newfile.txt','w')
for i in range(3000):
    f.write('hello world: ' + str(i) + '\n')

g = open('newfile.txt', 'r')
x=g.read()
print (x[:15*3-1],"\n……\n",x[-18*4:],sep="")
f.close()
g.close()

hello world: 0
hello world: 1
hello world: 2
……
hello world: 2209
hello world: 2210
hello world: 2211
hello world: 2212



In [48]:
import os
os.remove('newfile.txt')

出现异常时候的读写：

In [70]:
f = open('newfile.txt','w')
for i in range(3000):
    x = 1.0 / (i - 1000)
    f.write('hello world: ' + str(i) + '\n')

ZeroDivisionError: float division by zero

查看已有内容：

In [71]:
# 我们看看查看两次会导致的python优化
g = open('newfile.txt', 'r')
x = g.read()
print(x[:15*3-1], "\n……\n", x[-18*4:], sep="")
f.close()
g.close()

hello world: 0
hello world: 1
hello world: 2
……
456
hello world: 457
hello world: 458
hello world: 459
hello world: 460



In [72]:
g = open('newfile.txt', 'r')
x = g.read()
print(x[:15*3-1], "\n……\n", x[-18*4:], sep="")
f.close()
g.close()

hello world: 0
hello world: 1
hello world: 2
……
995
hello world: 996
hello world: 997
hello world: 998
hello world: 999



可以看到，出现异常的时候，磁盘的写入并没有完成，为此我们可以使用 `try/except/finally` 块来关闭文件，这里 `finally` 确保关闭文件，所有的写入已经完成。

In [73]:
f = open('newfile.txt','w')
try:
    for i in range(3000):
        x = 1.0 / (i - 1000)
        f.write('hello world: ' + str(i) + '\n')
except Exception:
    print ("something bad happened")
finally:
    f.close()

something bad happened


In [74]:
g = open('newfile.txt', 'r')
x = g.read()
print(x[:15*3-1], "\n……\n", x[-18*4:], sep="")
g.close()

hello world: 0
hello world: 1
hello world: 2
……
995
hello world: 996
hello world: 997
hello world: 998
hello world: 999



## with 方法

事实上，**Python**提供了更安全的方法，当 `with` 块的内容结束后，**Python**会自动调用它的`close` 方法，确保读写的安全：

In [75]:
with open('newfile.txt','w') as f:
    for i in range(3000):
        x = 1.0 / (i - 1000)
        f.write('hello world: ' + str(i) + '\n')

ZeroDivisionError: float division by zero

与 `try/exception/finally` 效果相同，但更简单。

In [76]:
g = open('newfile.txt', 'r')
x = g.read()
print(x[:15*3-1], "\n……\n", x[-18*4:], sep="")
g.close()

hello world: 0
hello world: 1
hello world: 2
……
995
hello world: 996
hello world: 997
hello world: 998
hello world: 999



所以，写文件时候要确保文件被正确关闭。

In [77]:
import os
os.remove('newfile.txt')